In [ ]:
#  Call Quality Analyzer
Assignment Submission- Voice AI Startup
Author:Muktha

This notebook analyzes sales call transcripts and extracts key conversational metrics.

Steps:

Input transcript lines with speaker labels (e.g., “Sales Rep:” and “Customer:”).
The system calculates:
Talk-time ratio (based on word count)
Number of questions (using “?” and question words)
Longest monologue (by word length)
Overall sentiment (positive, negative, or neutral using keyword counts)
One actionable insight (based on dominance in talk-time)
Why this works: This lightweight approach avoids heavy APIs and runs instantly in Jupyter, even on low-resource machines. It can be extended with real ASR (speech-to-text) for full automation.

In [5]:
import re
from collections import defaultdict
POSITIVE_WORDS ={"good","great","happy","excellent","love","awesome","nice","positive","thank","thanks"}
NEGATIVE_WORDS= {"bad","poor","angry","hate","awful","disappoint","problem","negative","issue","sorry"}

QUESTION_WORDS= r"^(who|what|when|where|why|how|is|are|do|does|did|can|could|would|should)\b"

def parse_plain_text(lines):
    utterances, last_speaker = [], None
    for line in lines:
        line=line.strip()
        if not line: 
            continue
        m = re.match(r'^\s*(?:\[[0-9:\.]+\]\s*)?([^:]+):\s*(.+)$', line)
        if m:
            speaker, txt = m.group(1).strip(), m.group(2).strip()
            utterances.append({"speaker": speaker,"text": txt,"words": len(txt.split())})
            last_speaker=speaker
        elif utterances:
            utterances[-1]['text'] += ' ' + line
            utterances[-1]['words']=len(utterances[-1]['text'].split())
    return utterances
def analyze(utterances):
    speaker_words=defaultdict(int)
    for u in utterances:
        speaker_words[u['speaker']]+=u.get('words',0)

    total=sum(speaker_words.values()) or 1
    talk_ratios={sp:round(speaker_words[sp]/total*100,2) for sp in speaker_words}

    questions, per_speaker_questions=0,defaultdict(int)
    longest, longest_speaker=0,None
    for u in utterances:
        txt=u['text']
        if '?' in txt or re.match(QUESTION_WORDS,txt,flags=re.I):
            questions+=1; per_speaker_questions[u['speaker']]+=1
        if u['words']>longest:
            longest=u['words']; longest_speaker=u['speaker']

    sentiment="Neutral"
    text=" ".join(u['text'] for u in utterances).lower()
    pos=sum(text.count(w) for w in POSITIVE_WORDS)
    neg=sum(text.count(w) for w in NEGATIVE_WORDS)
    if pos>neg: sentiment="Positive"
    elif neg>pos: sentiment="Negative"

    results={"talk_ratios":talk_ratios,
             "questions_total":questions,
             "questions_by_speaker":dict(per_speaker_questions),
             "longest_monologue":f"{longest} words by {longest_speaker}",
             "sentiment":sentiment,
             "insight":f"{max(talk_ratios,key=talk_ratios.get)} dominated the call. Try balancing conversation."}
    return results

In [7]:
demo_transcript = [
    "Sales Rep: Hi, thanks for joining the demo today.",
    "Customer: Hello, I wanted to know if this works with small teams?",
    "Sales Rep: Yes, absolutely! It’s designed for flexibility.",
    "Customer: Great, and what about the pricing?",
    "Sales Rep: Pricing is scalable based on your needs.",
    "Customer: Thanks, that’s helpful."
]
utterances = parse_plain_text(demo_transcript)
results = analyze(utterances)
results

{'talk_ratios': {'Sales Rep': 50.0, 'Customer': 50.0},
 'questions_total': 2,
 'questions_by_speaker': {'Customer': 2},
 'longest_monologue': '11 words by Customer',
 'sentiment': 'Positive',
 'insight': 'Sales Rep dominated the call. Try balancing conversation.'}